In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np

In [ ]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [ ]:
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 파일 경로
file_path = '/content/drive/MyDrive/PC - Desktop/2024-1. 안교수님-지식그래프-팀플/data/기록물.xlsx'
task_path = '/content/drive/MyDrive/PC - Desktop/2024-1. 안교수님-지식그래프-팀플/data/제주특별자치도개발공사 기록관리기준표(2023.12. 기준)_고시.xlsx'

In [ ]:
# 엑셀 파일 불러오기
df = pd.read_excel(file_path)

In [ ]:
task = pd.read_excel(task_path, header=1)

In [ ]:
df.head(2)

In [ ]:
print(df.shape)

In [ ]:
df_100 = df.iloc[:100, :]
print(df_100.shape)

In [ ]:
df_100.head(2)

In [ ]:
task.head(2)

In [ ]:
print(task.shape)

# 띄어쓰기 검사 후 교정
https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
from pykospacing import Spacing

# sentence = 'V리그 여자부 흥국생명이 김연경과 옐레나의 쌍포를 앞세워 가장 먼저 10승 고지를 밟았다. 흥국생명은 25일 홈경기에서 도로공사를 상대로 세트 스코어 3대0(27-25 25-20 25-19) 완승을 거뒀다.'
# new_sentence = sentence.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
# print(new_sentence)

In [ ]:
text = '마음온신례추가인물굼검사감독조서'
spacing = Spacing()
kospacing_sent = spacing(text)

print(kospacing_sent)

In [ ]:
# import dask.dataframe as dd
# import time

# # 시작 시간 기록
# start_time = time.time()

# # DataFrame을 Dask DataFrame으로 변환
# ddf = dd.from_pandas(df, npartitions=100)

# # apply 함수 적용
# ddf['제목_spacing'] = ddf['제목'].apply(spacing, meta=('제목_spacing', 'object'))

# # 결과를 다시 Pandas DataFrame으로 변환
# df = ddf.compute()

# # 종료 시간 기록
# end_time = time.time()

# # 소요 시간 출력
# print(f"Time taken: {end_time - start_time} seconds")

In [ ]:
# import re

# # 정규 표현식을 사용하여 글자가 아닌 문자 제거 함수 정의
# def remove_non_alphabetic(text):
#     # \w는 알파벳과 숫자, 밑줄(_)을 포함하므로, ^\w는 그 외의 문자를 의미
#     return re.sub(r'[^\w\s]', '', text)

# # '제목_spacing' 열에 함수 적용
# df['제목_spacing'] = df['제목_spacing'].apply(remove_non_alphabetic)

# # 결과 출력
# print(df['제목_spacing'].head(10))

In [ ]:

import re

# 정규 표현식을 사용하여 글자가 아닌 문자 제거 함수 정의
def remove_non_alphabetic(text):
    # \w는 알파벳과 숫자, 밑줄(_)을 포함하므로, ^\w는 그 외의 문자를 의미
    return re.sub(r'[^\w\s]', '', text)

# '제목' 열에 함수 적용
df['제목'] = df['제목'].apply(remove_non_alphabetic)

# 결과 출력
print(df['제목'].head(10))

# Finetuning
- 영어 bert 사용 클래스 뷴류(base-uncased)
https://github.com/leechanwoo-kor/bert-fine-tuning
- 한국어 bert 사용 클래스 분류(kykim/bert-kor-base)
https://teddylee777.github.io/huggingface/bert-kor-text-classification/#%EC%82%AC%EC%A0%84%ED%95%99%EC%8A%B5pre-trained-model-%EC%83%9D%EC%84%B1



### 1. 데이터셋 생성
- 제목_spacing : 맞춤법 검사 후 제목
- Label : 정수로 매핑된 단위과제명


In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
# # !pip install pykospacing

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, Dataset, RandomSampler
from tqdm import tqdm

In [ ]:
# 데이터프레임에서 제목_spacing와 단위업무명 열을 선택합니다.
# X = df['제목_spacing']
X = df['제목']
y = df['단위업무명']

print('X : ', X)
print('y: ', y)

X :  0                                       차량 배차 신청서
1                                        차량 운행 일지
2                         시설환경팀 12월 추가 교육시간 반영 요청
3         시설환경팀2024년도 삼다수사업장 공병부산물 매각 온비드 입찰 협조요청
4               4분기 삼다수사업장 폐수처리시설 방류수 수질검사 검사감독조서
                           ...                   
113672                       물류2팀 경조사 물품 지원 협조 요청
113673               수질안정성 확보를 위한  삼다수 품질검사 확대 계획
113674              2023년 삼다수사업장내 승강기 유지관리 용역 계획안
113675                       전산서비스요청서SMS 발송 권한 요청
113676                                    유연근무신청서
Name: 제목, Length: 113677, dtype: object
y:  0         삼다수사업장관용차량관리 
1         삼다수사업장관용차량관리 
2             부서교육및연수관리
3            삼다수사업장환경관리
4            삼다수사업장환경관리
              ...      
113672           부서서무업무
113673           부서서무업무
113674    삼다수사업장내부시설물관리
113675           제안제도운영
113676        부서인사및복무관리
Name: 단위업무명, Length: 113677, dtype: object


In [ ]:
# y 정수 레이블
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_map = encoder.fit_transform(y)
y_map_max = y_map.max() + 1

df['Label'] = y_map
print(df['Label'])
print(y_map_max)

0         326
1         326
2         255
3         367
4         367
         ... 
113672    263
113673    263
113674    329
113675    584
113676    267
Name: Label, Length: 113677, dtype: int64
704


In [ ]:
dataset = df[['제목', 'Label']]
dataset.head()

,제목,Label
0,차량 배차 신청서,326
1,차량 운행 일지,326
2,시설환경팀 12월 추가 교육시간 반영 요청,255
3,시설환경팀2024년도 삼다수사업장 공병부산물 매각 온비드 입찰 협조요청,367
4,4분기 삼다수사업장 폐수처리시설 방류수 수질검사 검사감독조서,367


### 토크나이즈 적용
- 한국어용 모델 : 'kykim/bert-kor-base'
- 참고 링크 : https://huggingface.co/kykim/bert-kor-base

In [ ]:
import torch

# 사용 가능한 GPU 수 확인
num_gpus = torch.cuda.device_count()
print(f"사용 가능한 GPU 수: {num_gpus}")
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# GPU가 있으면 첫 번째 GPU를 사용하고, 없으면 CPU를 사용
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"사용할 장치: {device}")

사용 가능한 GPU 수: 1
GPU 0: Tesla V100-SXM2-16GB
사용할 장치: cuda:0


In [ ]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'

In [ ]:
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader


class TokenDataset(Dataset):

    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe
        # Huggingface 토크나이저 생성
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['제목']
        label = self.data.iloc[idx]['Label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)


In [ ]:
import torch
from torch.utils.data import random_split, DataLoader

# 토크나이저 지정
tokenizer_pretrained = CHECKPOINT_NAME

# train, test 데이터셋 생성
token_data = TokenDataset(dataset, tokenizer_pretrained)

# 데이터셋 크기 계산
dataset_size = len(token_data)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

# 데이터셋을 8:2로 분할
train_dataset, test_dataset = random_split(token_data, [train_size, test_size])

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8)

# 확인용 출력
print(f"Train DataLoader size: {len(train_loader)}")
print(f"Test DataLoader size: {len(test_loader)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Train DataLoader size: 5684
Test DataLoader size: 1421


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))

# 데이터셋을 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


tensor([207, 160, 253, 267, 119,  19, 228, 570, 507, 701, 500, 398, 485, 263,
        305, 491], device='cuda:0')

In [ ]:
# 생성된 inputs의 key 값 출력
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'token_type_ids'])

In [ ]:
# key 별 shape 확인
inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape

(torch.Size([16, 512]), torch.Size([16, 512]), torch.Size([16, 512]))

In [ ]:
from transformers import BertConfig

config = BertConfig.from_pretrained(CHECKPOINT_NAME)
config

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "embedding_size": 768,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

# 사전학습 model 생성

In [ ]:
from transformers import BertModel

# 모델 생성
model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)
model_bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
output = model_bert(**inputs)
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
output['last_hidden_state'].shape, output['pooler_output'].shape

(torch.Size([16, 512, 768]), torch.Size([16, 768]))

In [ ]:
# last_hidden_state 출력
last_hidden_state = output['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])

torch.Size([16, 512, 768])
tensor([[ 3.8871e-01, -5.9039e-01,  4.4721e-01,  ...,  2.6780e-04,
         -9.5683e-01,  1.8466e-01],
        [ 2.6873e-01, -4.0486e-01,  6.8391e-01,  ...,  3.2373e-01,
         -7.7215e-01,  4.0058e-01],
        [ 3.7875e-01, -2.5190e-01,  9.7175e-01,  ...,  6.9341e-02,
         -6.0687e-01,  5.0730e-01],
        ...,
        [ 9.2019e-01, -4.0593e-01,  6.9313e-01,  ..., -3.9296e-02,
         -9.1677e-01,  5.4059e-01],
        [ 4.5912e-01, -1.9004e-01, -3.5425e-01,  ...,  2.6354e-01,
         -1.2402e+00,  7.2750e-01],
        [ 1.8067e-01, -3.1871e-01,  5.0921e-01,  ...,  1.2069e-01,
         -4.1349e-01,  4.0194e-01]], device='cuda:0', grad_fn=<SliceBackward0>)


In [ ]:
# pooler_output 출력
pooler_output = output['pooler_output']
print(pooler_output.shape)
print(pooler_output)

torch.Size([16, 768])
tensor([[-0.9141,  0.2514, -0.7185,  ..., -0.4059,  0.1512,  0.2169],
        [-0.8059,  0.2942, -0.7538,  ...,  0.7003,  0.0486,  0.4469],
        [-0.7007,  0.1919, -0.8690,  ...,  0.5833,  0.2241,  0.4294],
        ...,
        [-0.8880,  0.1629, -0.8924,  ..., -0.2597,  0.2613,  0.7956],
        [-0.9269,  0.4385, -0.8169,  ...,  0.5280, -0.0887,  0.5243],
        [-0.6584,  0.2344, -0.6547,  ...,  0.4626,  0.1339,  0.2584]],
       device='cuda:0', grad_fn=<TanhBackward0>)


In [ ]:
import torch.nn as nn

fc = nn.Linear(768, y_map_max)
fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))

torch.Size([16, 704])
tensor([364, 205,  49,  49,  49,  49,  49,  49,  49, 205, 205,  49,  49,  49,
         49,  49], device='cuda:0')


In [ ]:
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, y_map_max)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [ ]:
# CustomBertModel 생성
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)

CustomBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
from torch import optim
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

In [ ]:
from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()

    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()

        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)

        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)

        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()

        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()

        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)

        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(labels).sum().item()
        counts += len(labels)

        # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)

        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")

    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [ ]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()

    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs)

            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)

            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(labels)).item()

            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)

        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [ ]:
# 최대 Epoch을 지정합니다.
num_epochs = 3

# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'bert-kor-base'

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)

    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), f'{model_name}.pth')

    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

training loss: 52.22942, training accuracy: 0.40379: 100%|██████████| 5684/5684 [41:15<00:00,  2.30batch/s]


[INFO] val_loss has been improved from inf to 1.97731. Saving Model!
epoch 01, loss: 3.26445, acc: 0.40379, val_loss: 1.97731, val_accuracy: 0.58889


training loss: 28.96216, training accuracy: 0.62800: 100%|██████████| 5684/5684 [41:16<00:00,  2.29batch/s]


[INFO] val_loss has been improved from 1.97731 to 1.39890. Saving Model!
epoch 02, loss: 1.81019, acc: 0.62800, val_loss: 1.39890, val_accuracy: 0.68882


training loss: 21.12329, training accuracy: 0.71288: 100%|██████████| 5684/5684 [41:12<00:00,  2.30batch/s]


[INFO] val_loss has been improved from 1.39890 to 1.13203. Saving Model!
epoch 03, loss: 1.32025, acc: 0.71288, val_loss: 1.13203, val_accuracy: 0.73817


In [ ]:
# 저장한 state_dict를 로드 합니다.
bert.load_state_dict(torch.load(f'{model_name}.pth'))

<All keys matched successfully>

In [ ]:
class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels

    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        return prob, result

In [ ]:
# Huggingface 토크나이저 생성
tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)

# label_map 생성
label_map = {index: label for index, label in enumerate(encoder.classes_)}

# labels 딕셔너리 생성
labels_dict  = {i: label for i, label in label_map.items()}
print(labels_dict )

# CustomPredictor 인스턴스를 생성합니다.
predictor = CustomPredictor(bert, tokenizer, labels_dict )

{0: '      정부권장정책 공공구매', 1: ' CCM일반관리활동', 2: ' 공사디자인(CI)운영', 3: '442안전점검운영', 4: '4대보험관리', 5: '4대폭력예방업무', 6: 'A지구신규타당성조사업무', 7: 'CCM인증취득심사관리', 8: 'CEO지시사항관리', 9: 'CIP운영관리', 10: 'CSR심의위원회운영및관리', 11: 'CS교육기획및운영', 12: 'CS협의체운영', 13: 'ERP시스템관리', 14: 'FSSC22000인증관리', 15: 'HACCP인증관리', 16: 'ISO14001인증관리 ', 17: 'ISO22301재해경감우수기업인증', 18: 'ISO9001인증관리', 19: 'JPDC 품질문화 활동', 20: 'L6신규생산설비 도입(공장 자동화)', 21: 'L6신규생산설비 도입(공통)', 22: 'L6신규생산설비 도입(사출)', 23: 'L6신규생산설비 도입(정수)', 24: 'L6신규생산설비 도입(충전포장)', 25: 'L6신규생산설비도입(유틸리티 배관공사)', 26: 'L6신규생산설비도입(유틸리티 부대공사)', 27: 'MRO운영관리', 28: 'NSF인증관리', 29: 'SNS 채널콘텐츠제작관리', 30: 'VOC통합관리기획', 31: '가공용감귤수매가정산지출', 32: '가공용감귤수매관리', 33: '각종분담금신고납부', 34: '감귤1공장지하수관리및운영', 35: '감귤가공관리', 36: '감귤가공산업안전관리', 37: '감귤공장가동계획', 38: '감귤공장구내식당 운영', 39: '감귤공장보안관리', 40: '감귤공장사업장오?폐수처리장관리', 41: '감귤공장사업장폐기물관리', 42: '감귤공장설비점검및유지관리', 43: '감귤공장소방안전관리', 44: '감귤공장시설물관리', 45: '감귤공장안전보건활동대응', 46: '감귤공장운영정비점검', 47: '감귤공장위험물관리', 48: '감귤공장인력운영', 49: '감귤공장인스펙션(불시점검)대응', 50: '감귤공장인증심사대응', 51: '감귤공장품질관리', 5

In [ ]:
import torch.nn.functional as F

# def predict_sentence(predictor):
#     input_sentence = input('문장을 입력해 주세요: ')
#     predictor.predict(input_sentence)

# predict_sentence(predictor)

In [ ]:
# 테스트 데이터셋에 대해 예측 수행 및 결과 출력
test_sentences = [test_dataset[i][0]['input_ids'] for i in range(len(test_dataset))]
test_true_labels = [test_dataset[i][1].item() for i in range(len(test_dataset))]

true_list = []
pred_list = []
for i in range(len(test_sentences)):
    sentence = tokenizer.decode(test_sentences[i], skip_special_tokens=True)
    prob, predicted_label = predictor.predict(sentence)
    true_label = labels_dict.get(test_true_labels[i], "Unknown")

    # print(f"Sentence: {sentence}")
    # print(f"True Label: {true_label}")
    # print(f"True predicted_label: {predicted_label}")
    # print(f"prop: {prop}")

    true_list.append(true_label)
    pred_list.append(predicted_label)
print(len(true_list), len(pred_list))

22736 22736


In [ ]:
count = 0
for i in range(len(true_list)):
    if true_list[i] == pred_list[i]:
        count += 1

print(f'같은 값을 가진 인덱스의 수: {count}')


# 정확도 계산
accuracy = count / len(true_list)

print("정확도:", accuracy)
print(f'맞춘 개수 / 전체 개수 : {count}/{len(true_list)}')

같은 값을 가진 인덱스의 수: 16775
정확도: 0.7378166783954961
맞춘 개수 / 전체 개수 : 16775/22736
